In [96]:
import time
import selenium.webdriver
from bs4 import BeautifulSoup
import itertools
import pandas as pd
import numpy as np
import re
from selenium.common.exceptions import NoSuchElementException

with open('us_users_urls.csv') as f:
    for line in f:
        us_user_urls = line.split(',')
for i, user in enumerate(us_user_urls):
    us_user_urls[i] = str(user.strip('\"'))

def get_boardgamegeek_user_info(query, browser, delay=5):
    user_info = []
    search_url = "https://boardgamegeek.com/collection{}?rated=1&subtype=boardgame&ff=1".format(query)
    browser.get(search_url)
    while True:
        html = browser.page_source
        soup = BeautifulSoup(html, 'html.parser')
        product_tags = soup.find_all('tr', {"id" : lambda x: x and x.startswith('row')})
        user_info.append([get_user_info(t) for t in product_tags])
        try:
            browser.find_element_by_link_text('»').click()
        except NoSuchElementException:
            html = browser.page_source
            soup = BeautifulSoup(html, 'html.parser')
            product_tags = soup.find_all('tr', {"id" : lambda x: x and x.startswith('row')})
            user_info.append([get_user_info(t) for t in product_tags])
            break
    time.sleep(delay)
    ratings = []
    for i in xrange(1, len(user_info)):
        ratings += user_info[i]
    return list(set(ratings))

def get_user_info(product_tag):
    title_elements = product_tag.find_all("div")
    if title_elements:
        rated_games = title_elements[1].text.split()[:-1]
        rated_games = ' '.join(rated_games)
        if rated_games == u'':
            rated_games = title_elements[1].text.split()
            rated_games = ' '.join(rated_games)
        ratings = title_elements[7].text.split()[0]
        date = ' '.join(title_elements[8].text.split()[0:2])
        date = date.strip('*')
        return rated_games, float(ratings), date
    
browser = selenium.webdriver.PhantomJS(executable_path='/Users/ericyatskowitz/galvanize_work/capstone/phantomjs-2.1.1-macosx/bin/phantomjs')
user_info = {}
for url in us_user_urls[0:30]:
    user = url.split('/')[2]
    user_ratings = get_boardgamegeek_user_info(url, browser)
    user_info[user] = user_ratings
    time.sleep(5)
browser.quit()

np.save('us_ratings_data.npy', user_info)

In [97]:
read_dictionary = np.load('us_ratings_data.npy').item()

In [100]:
read_dictionary

{'2ndPlace': [(u'Coloretto', 7.0, u'Jun 2004'),
  (u'Royal Palace', 8.8, u'May 2009'),
  (u'Canasta Caliente', 7.5, u'Jul 2005'),
  (u'Key Harvest', 7.0, u'Oct 2012'),
  (u'Fast Flowing Forest Fellers', 7.0, u'Jul 2009'),
  (u'Maya', 8.0, u'Sep 2004'),
  (u'Gemblo', 9.0, u'Mar 2006'),
  (u"Aladdin's Dragons", 8.0, u'Jun 2004'),
  (u'Elfenland', 7.0, u'Jun 2004'),
  (u'Blokus 3D', 8.0, u'May 2005'),
  (u'Power Struggle', 9.0, u'Jul 2010'),
  (u'Modern Art', 9.0, u'May 2005'),
  (u'Nefertiti', 8.5, u'Oct 2008'),
  (u'Alien Frontiers', 9.0, u'May 2011'),
  (u'Takenoko', 8.3, u'Aug 2012'),
  (u'Pickomino', 7.5, u'Apr 2006'),
  (u'Scrabble', 7.0, u'Jun 2004'),
  (u'Forbidden Island', 9.0, u'Jun 2010'),
  (u'The Caucasus Campaign', 9.0, u'Jan 2010'),
  (u'Gang of Four', 8.0, u'May 2005'),
  (u'Castle Panic', 9.0, u'Jan 2012'),
  (u'Mammoth Hunters', 7.0, u'Dec 2008'),
  (u'Ghost Stories', 8.0, u'Jan 2009'),
  (u'Alexander the Great', 8.0, u'Feb 2010'),
  (u'Manoeuvre', 8.0, u'Sep 2009'),
  (